In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile
from selenium.webdriver.common.by import By
import time
import regex as re
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import ElementClickInterceptedException

import pandas as pd
import threading
import sys,os
import re
import json
sys.path.append("C:\\Users\\7ross\\Desktop\\ishanu scripts\\repo\\recogai_\\nero\\nero-data")
import utils
from collections import deque

In [2]:
# natural sorting implementation 
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

In [3]:
js_sem = threading.Semaphore(1)
pool_sem = threading.Semaphore(1)

browser_pool = deque()

in_dir = r"C:\Users\7ross\Desktop\ishanu scripts\repo\recogai_\nero\nero-data\api_data_collection\openai\gpt4.0\generated_text"
json_name = "gpt_4zero_detection.json"
d = {}

if(json_name in os.listdir(os.getcwd())):
    with open(json_name,"r") as infile:
        data = json.load(infile)
        d = data
    




In [4]:
options = Options()
firefox_profile = FirefoxProfile()
firefox_profile.set_preference("javascript.enabled", True)
options.profile = firefox_profile
utils.max_threads = 5
for i in range(utils.max_threads):
    h_driver = webdriver.Firefox(options=options)
    h_driver.get("https://howkgpt.nyuad.nyu.edu/")
    time.sleep(3)


    z_driver = webdriver.Firefox(options=options)
    z_driver.install_addon("uBlock0_1.64.1b8.firefox.signed.xpi")
    z_driver.get("https://www.zerogpt.com/")
    time.sleep(3)
    br_coup = (h_driver,z_driver)
    browser_pool.append(br_coup)

In [5]:
@utils.multithreading
def get_howk_and_zero(br_coup,entry):
    global d
    global json_name
    global browser_pool
    idx = re.findall(r"\d+",entry.name)[-1]

    text = ""
    with open(entry.path,"r",encoding="utf-8") as infile:
        text = infile.read()
    text = text[int(len(text)/2):int(len(text)/2)+15000:1]



    pred = ""
    per = 0.0
    perp = 0.0
    h_driver = br_coup[0]
    z_driver = br_coup[1]
    
    text_area = h_driver.find_element(By.ID,"text")
    button = h_driver.find_element(By.ID,"check")
    text_area.clear()
    text_area.send_keys(text)
    loops = 0
    while(True):
        loops +=1
        if(loops == 24):
            print(f"howkgpt timed out")
            with pool_sem:
                browser_pool.append(br_coup)
            return pred, perp, per
        time.sleep(0.250)
        try:
            button.click()
            break
        except ElementClickInterceptedException:
        
            continue
    
    pred_paragraph = None
    loops = 0
    while(True):
        loops += 1
        if(loops == 12):
            print(f"howkgpt timed out")
            with pool_sem:
                browser_pool.append(br_coup)
            return pred, perp, per
        time.sleep(0.5)
        
        try:
            pred_paragraph = h_driver.find_element(By.XPATH,"//p[contains(text(), 'Text created by:')]")
            break
        except NoSuchElementException:
            
            continue
        except WebDriverException as e:
            with pool_sem:
                browser_pool.append(br_coup)
            return pred,perp,per
    
    pred = pred_paragraph.text.replace("Text created by: ","")
    perp_paragraph = h_driver.find_element(By.XPATH,"//p[contains(text(), 'Perplexity Score:')]")
    perp = re.search(r"[-+]?\d*\.?\d+",perp_paragraph.text).group(0)
    

    z_text_area = z_driver.find_element(By.ID,"textArea")
    z_button = z_driver.find_element(By.CLASS_NAME,"scoreButton")
    z_text_area.clear()
    z_text_area.send_keys(text)
    while(True):
        time.sleep(0.250)
        try:
            z_button.click()
            break
        except ElementClickInterceptedException:
            print(f"click error")
            continue
    percentage = None
    while(True):
        time.sleep(0.5)
        try:
            percentage = z_driver.find_element(By.XPATH,"//span[contains(.,'%')]")
            break
        except NoSuchElementException:
            continue
        
       

    per = re.search(r"[-+]?\d*\.?\d+",percentage.text).group(0)
    
    print(f"outputs ----------")
    print(f"prediction: {pred}")
    print(f"perplexity: {perp}")
    print(f"percentage: {per}")
    with js_sem:
        data = {"prediction":pred,"perplexity":perp,"percentage":per}
        d[idx] = data
        with open(json_name,"w") as outfile:
            json.dump(d,outfile,indent = 4)
            print(f"total current novels: {len(d)}")
        print(f"successfully dumped to json")
    with pool_sem:
        browser_pool.append(br_coup)
    return pred, perp, per

In [ ]:





with os.scandir(in_dir) as entries:
    for entry in entries:
        idx = re.findall(r"\d+",entry.name)[-1]
        print(f"entry: {entry}")
        #if the data json already has an entry for this file
        if(idx in d.keys()):
            #then if it actually has data skip
            continue

        while(len(browser_pool)<1):
            print(f"pool empty")
            time.sleep(1)
        with pool_sem:
            br_coup = browser_pool.pop()
        get_howk_and_zero(br_coup,entry)
        
for coup in browser_pool:
    coup[0].close()
    coup[1].close()     

entry: <DirEntry 'gpt_4zero0.txt'>
multithread threding count: 4
entry: <DirEntry 'gpt_4zero1.txt'>
multithread threding count: 3
entry: <DirEntry 'gpt_4zero10.txt'>
multithread threding count: 2
entry: <DirEntry 'gpt_4zero11.txt'>
multithread threding count: 1
entry: <DirEntry 'gpt_4zero12.txt'>
multithread threding count: 0
entry: <DirEntry 'gpt_4zero13.txt'>
pool empty
pool empty
pool empty
pool empty
pool empty
pool empty
pool empty
pool empty
pool empty
pool empty
pool empty
pool empty
pool empty
pool empty
pool empty
pool empty
pool empty
pool empty
pool empty
pool empty
pool empty
pool empty
pool empty
pool empty
pool empty
pool empty
pool empty
pool empty
outputs ----------
prediction: Human
perplexity: 37.59
percentage: 98.21
total current novels: 1
successfully dumped to json
multithread threding count: 0
entry: <DirEntry 'gpt_4zero14.txt'>
pool empty
outputs ----------
prediction: Human
perplexity: 46.7
percentage: 98.27
total current novels: 2
successfully dumped to json
mu

outputs ----------
prediction: AI
perplexity: 8.94
percentage: 78.9
total current novels: 46
successfully dumped to json
outputs ----------
prediction: Human
perplexity: 34.65
percentage: 90.72
total current novels: 47
successfully dumped to json
outputs ----------
prediction: Human
perplexity: 55.52
percentage: 80.84
total current novels: 48
successfully dumped to json
outputs ----------
prediction: AI
perplexity: 9.54
percentage: 88.3
total current novels: 49
successfully dumped to json
